In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
jm1 = pd.read_csv(r'/content/jm1_csv.csv')
jm1=jm1.replace('?',0)
print('jm1 shape: ', jm1.shape)
jm1.head()

jm1 shape:  (10885, 22)


,loc,v(g),ev(g),iv(g),n,v,l,d,i,e,...,lOCode,lOComment,lOBlank,locCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,defects
0,1.1,1.4,1.4,1.4,1.3,1.30,1.30,1.30,1.30,1.30,...,2,2,2,2,1.2,1.2,1.2,1.2,1.4,False
1,1.0,1.0,1.0,1.0,1.0,1.00,1.00,1.00,1.00,1.00,...,1,1,1,1,1,1,1,1,1,True
2,72.0,7.0,1.0,6.0,198.0,1134.13,0.05,20.31,55.85,23029.10,...,51,10,8,1,17,36,112,86,13,True
3,190.0,3.0,1.0,3.0,600.0,4348.76,0.06,17.06,254.87,74202.67,...,129,29,28,2,17,135,329,271,5,True
4,37.0,4.0,1.0,4.0,126.0,599.12,0.06,17.19,34.86,10297.30,...,28,1,6,0,11,16,76,50,7,True


In [ ]:
jm1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10885 entries, 0 to 10884
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   loc                10885 non-null  float64
 1   v(g)               10885 non-null  float64
 2   ev(g)              10885 non-null  float64
 3   iv(g)              10885 non-null  float64
 4   n                  10885 non-null  float64
 5   v                  10885 non-null  float64
 6   l                  10885 non-null  float64
 7   d                  10885 non-null  float64
 8   i                  10885 non-null  float64
 9   e                  10885 non-null  float64
 10  b                  10885 non-null  float64
 11  t                  10885 non-null  float64
 12  lOCode             10885 non-null  int64  
 13  lOComment          10885 non-null  int64  
 14  lOBlank            10885 non-null  int64  
 15  locCodeAndComment  10885 non-null  int64  
 16  uniq_Op            108

In [ ]:
target = 'defects'
y = jm1[target]
x=jm1.drop(['defects'], axis=1)
x.shape

(10885, 21)

In [ ]:
y.shape

(10885,)

In [ ]:
from sklearn.feature_selection import SelectKBest,  chi2
selector = SelectKBest(chi2, k=5)

x_reduced = selector.fit_transform(x,y)
x_reduced.shape



(10885, 5)

In [ ]:
cols = selector.get_support(indices=True)
selected_columns = x.iloc[:,cols].columns.tolist()
selected_columns

['n', 'v', 'e', 't', 'total_Op']

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_reduced, y, test_size=0.3, random_state=10)

In [ ]:
print(x.shape)
print(y.shape)
print('-------------------')
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(10885, 21)
(10885,)
-------------------
(7619, 5)
(7619,)
(3266, 5)
(3266,)


In [ ]:
from sklearn.svm import SVC
from sklearn import metrics #for checking the model accuracy
svc = SVC()
svc.fit(x_train, y_train)
svc_pred=svc.predict(x_test)
from sklearn import metrics #for checking the model accuracy
acc_svm = metrics.accuracy_score(svc_pred,y_test)
print('The accuracy of the SVM is:',acc_svm)
print('Th confusion matrix of SVM:\n ',metrics.confusion_matrix(y_test, svc_pred))

The accuracy of the SVM is: 0.8000612369871403
Th confusion matrix of SVM:
  [[2603    7]
 [ 646   10]]


In [ ]:
print(metrics.confusion_matrix(y_test, svc_pred))
print(metrics.classification_report(y_test, svc_pred))

[[2603    7]
 [ 646   10]]
              precision    recall  f1-score   support

       False       0.80      1.00      0.89      2610
        True       0.59      0.02      0.03       656

    accuracy                           0.80      3266
   macro avg       0.69      0.51      0.46      3266
weighted avg       0.76      0.80      0.72      3266



In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=2) #put 2 neighbors because we have 2 labels.
knn.fit(x_train,y_train)
knn_pred = knn.predict(x_test)
acc_knn = metrics.accuracy_score(knn_pred,y_test)
print('The accuracy of the KNN is', acc_knn)
print('The confusion matrix of KNN:\n ',metrics.confusion_matrix(y_test, knn_pred))
print(metrics.confusion_matrix(y_test, knn_pred))
print(metrics.classification_report(y_test, knn_pred))

The accuracy of the KNN is 0.7890385793018984
The confusion matrix of KNN:
  [[2515   95]
 [ 594   62]]
[[2515   95]
 [ 594   62]]
              precision    recall  f1-score   support

       False       0.81      0.96      0.88      2610
        True       0.39      0.09      0.15       656

    accuracy                           0.79      3266
   macro avg       0.60      0.53      0.52      3266
weighted avg       0.73      0.79      0.73      3266



In [ ]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
model.fit(x_train, y_train)
y_predict = model.predict(x_test)
model_score = model.score(x_test, y_test)
print(model_score)
print(metrics.confusion_matrix(y_test, y_predict))
print(metrics.classification_report(y_test, y_predict))

0.7526025719534599
[[2290  320]
 [ 488  168]]
              precision    recall  f1-score   support

       False       0.82      0.88      0.85      2610
        True       0.34      0.26      0.29       656

    accuracy                           0.75      3266
   macro avg       0.58      0.57      0.57      3266
weighted avg       0.73      0.75      0.74      3266



In [ ]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(x_train, y_train)
model.score(x_train, y_train) 

0.8094238088988056

In [ ]:
test_pred = model.predict(x_test)
print(metrics.classification_report(y_test, test_pred))
print(metrics.confusion_matrix(y_test, test_pred))

              precision    recall  f1-score   support

       False       0.81      0.97      0.88      2610
        True       0.46      0.10      0.16       656

    accuracy                           0.80      3266
   macro avg       0.63      0.54      0.52      3266
weighted avg       0.74      0.80      0.74      3266

[[2531   79]
 [ 590   66]]
